<a href="https://colab.research.google.com/github/fisherj1/Neural/blob/Lesson8/Lesson_08_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерация текста с использованием LSTM сетей
Импортируем numpy, чтобы преобразовывать буквы в числа, да и вообще это must-have. tensorflow это понятно, а os будет нужен, чтобы работать с путями к файлам.

In [ ]:
import tensorflow as tf
import numpy as np
import os
import urllib

In [ ]:
os.mkdir("input_data/")

In [ ]:
resource = urllib.request.urlopen("http://t.stelm.ru/nn/lesson7/frankenstein.txt")
out = open("input_data/frankenstein.txt", 'wb')
out.write(resource.read())
out.close()

Далее откроем локально приложенный текст, на котором мы будем обучаться. Приводится текст на английском языке, что влияет на коды букв и фильтрацию лишних символов. При открытии текстового файла надо учитывать кодировку. Нельзя просто считать один байт одной буквой. Некоторые кодировки могут обозначать буквы двумя байтами. Проще всего выбрать кодировку UTF-8, убедиться в текстовом редакторе, что файл сохранён в ней и использовать эту кодировку при открытии файла.

In [ ]:
path_to_file = "input_data/frankenstein.txt"

# Открыть файл на чтение в бинарном виде, считать и декодировать
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

### Фикс для GPU
Для видеокарт с 4 ГБ памяти и меньше tensorflow оставляет недостаточный запас свободной видеопамяти, поэтому включим выделение памяти по требованию, вместо дефолтной процеду выделения сразу почти всей памяти.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

### Проверим текст
Мы ожидаем, что текст корректно считался, что в нём менее 80 символов (для текстов на английском), включающих в себя строчные и прописные буквы (26*2 для английского алфавита),  пунктуацию(18), символы переноса строки (2). Для проверки текста сделаем функцию, которая выведет начало текста на экран, посчитает количество символов в нём, посчитаем уникальные символы и выведем их. Также сразу выделим символы за пределами стандартного алфавита, цифр и пунктуации.

In [ ]:
def check_text(text):
    # Посмотрим первые 300 символов
    print(text[:300])
    # Посмотрим общее количество символов
    print ('\nLength of text: {} characters'.format(len(text)))
    # Построим перечень уникальных символов, а numpy поможет нам работать с массивом
    vocab = np.array(sorted(set(text)))
    # Выведем на экран все уникальные символы
    print ('{} unique characters:'.format(len(vocab)))
    print(vocab)
    # Нестандартные специальные символы в английском начинаются после символа 'z'
    print ('Bad characters in english text:')
    print(vocab[vocab > 'z'])
    return vocab

vocab = check_text(text)

Если в тексте есть плохие символы (нестандартные специальные символы), то стоит их отфильтровать из текста. Для этого берем последний символ, после которого начинаются специальные символы и уберем их из текста. Это не совсем корректно, ведь это могли быть, например, буквы с поставленными ударениями, которые нужно заменить на буквы без ударений. Но для простоты и универсальности - просто удалим лишнее.

In [ ]:
# Удаляем символы после последнего значимого
text = text.translate({ord(c): None for c in vocab[vocab > 'z']}) # As recommended in https://stackoverflow.com/questions/3939361/remove-specific-characters-from-a-string-in-python
# Снова смотрим на текст, на словарь и пересохраняем его себе
vocab = check_text(text)

### Перевод символов в числа и наоборот

Теперь текст должен состоять только из хороших символов и можно продолжать. Но для нейронной сети нам нужны числа, а не символы. Сделаем это с помощью двух таблиц-словарей. Вообще одна таблица у нас уже есть - наш словарь символов. По индексу он выдаёт символ. Можно сделать обратный к нему словарь. Далее переведем текст в целые числа (индексы словаря). В конце еще сделаем тип массива этих чисел int8, так как у нас небольшой словарь, а по умолчанию будет использоваться int64, что в 8 раз больше тратит памяти.

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = vocab

text_as_int = np.array([char2idx[c] for c in text]).astype('uint8')

Проверим, что текст в виде индексов соответствует тексту в виде символов. Для этого выведем их вместе.

In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

### Создание обучающей выборки

Наша цель в обучающей выборке это выделять из текста пары строк определённой длины (`seq_length`), где вторая строка это первая, сдвинутая на один символ вперед. Первая строка это вход, а вторая это ожидаемый выход, который должен уметь предсказывать текст на один символ вперед. Так мы разобьём весь текст. Для этого мы будем использовать `seq_length+1`, чтобы эта длина покрывала и входную строку, и выходную.
Воспользуемся высокоуровневыми методами создания обучающей выборки. Для этого входной массив преобразуем в Dataset с помощью `tf.data.Dataset.from_tensor_slices`. Раньше мы имели список символов, а теперь имеем специальный объект-тензор, где внутри тоже список символов. Чтобы достать данные из тензора надо прочитать правила работы с датасетами: https://www.tensorflow.org/guide/data. Там приводится метод enumerate, которым мы воспользуемся как обычным enumerate в Питоне. Другие методы не заработали, так как Dataset создан для работы с потоками данных и не имеет самих данных, чтобы о них что-то рассказать. Не забудем получающиеся тензоры конвертировать в числа с помощью метода numpy() и затем переводить в буквы.

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
print("There will be {} examples".format(examples_per_epoch))

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
print("Let's check tensor properties: {}".format(char_dataset))

# Enumerate method
for i,v in char_dataset.enumerate().take(7):
    print (idx2char[v.numpy()])
# Simplier method
for v in char_dataset.take(7):
    print (idx2char[v.numpy()])

Теперь единый тензор разобьём на набор тензоров с точками разбиения на позиции `seq_length+1`. Отбросим хвост текста - не жалко, зато данные выровнены. Опять же пока мы не попытаемся перебрать элементы тензора, тензор о своих данных  ничего не сообщит, хотя теперь он знает одну из размерностей - `seq_length+1`, ведь мы бьём текст именно на такие отрезки.
Проверим, что первый элемент это строка, начинающаяся с начала текста, и что нет пропущенных мест при разбиении на подстрочки. Для этого воспользуемся тем, что наш словарь для перевода текста в индексы и обратно может принимать на вход массив и возвращать массив. Поэтому каждая строка это массив символов.

In [ ]:
# Slicing dataset
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Now we can check the properties
print("Let's check tensor properties: {}".format(sequences))

# Visualize first 2 sequences translating them to letters
for item in sequences.take(2):
    print (idx2char[item.numpy()])

Здесь мы перебрали элементы тензора и для каждого сконвертировали его в массив numpy, который надо было перевести в символы. Однако так читать текст неудобно, поэтому можно дополнительно соединить символы методом `join`. Сделаем функцию для вывода строки из тензора с индексами символов и проверим эту функцию.

In [ ]:
def show_str(item, prefix=None):
    if prefix is not None:
       print(prefix)
    print("".join(idx2char[item]))

# Visualize first 3 sequences
for i,v in sequences.enumerate().take(3):
    show_str(v, "Sequence {}:".format(i))

Теперь видно еще лучше, что текст разбит на строки, где каждая следующая продолжает предыдущую.

Для обучения нам нужны пары строк, сдвинутых на один символ, а не просто много строк из текста. Для этого есть метод map(), который каждый входной элемент массива-тензора заменит на выходной элемент, который ему вернёт некая функция. Эту функцию `split_input_target()` пишем мы сами.

In [ ]:
# Define splitting method
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Test splitting method
print(split_input_target("Test phrase"))

# Creating dataset from sequences with our splitting method
dataset = sequences.map(split_input_target)

Проверим что получилось в нашем датасете с помощью уже знакомых методов перебора элементов тензора. Посмотрим первые 2 пары пар строк для обучения.

In [ ]:
for input_example, target_example in dataset.take(2):
    show_str (input_example, 'Input data: ')
    show_str (target_example, 'Target data: ')

Каждая пара строк в обучающей выборке подаётся посимвольно в нейросеть. Каждый шаг по времени t подаётся символ c индексом t входной строки и ожидается на выходе символ c индексом t выходной строки.
Для этого объединим строки в кортеж. В строках возьмём только первые 5 символов. Кортеж сделаем с возможностью итерации с помощью `zip()`. Теперь можно способом `enumerate()` перебрать все пары символов (которые на самом деле индексы в словаре символов). Видно, что для каждого символа мы ждём следующий символ на выходе. И так `seq_length` раз, но мы визуализируем только первые 5.

In [ ]:
# Take first dataset string pair again
for input_example, target_example in dataset.take(1):
    pass
# Show symbols for each time step as illustration
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, idx2char[input_idx]))
    print("  expected output: {} ({:s})".format(target_idx, idx2char[target_idx]))

### Создание обучающих пакетов

Пары строк в нашем датасете мы разобьём на пакеты (batch) для ускорения обучения. То есть вместо подачи каждой пары строк пока эти пары не кончатся и не пройдёт эпоха, мы будем подавать сразу пачку строк и после каждой пачки подстраивать сеть. Возникает вопрос, какой должен быть размер батча? Крайние случаи это батч из одного элемента и батч из всех элементов. Подстраивать сеть каждую пару строк будет приводить к метаниям градиента, ведь в каждой конретной паре строк свои правила предсказания буквы. Если всё подавать единым батчем, то может не хватить памяти в ускорителе, да и подстраивать коэффициенты раз за эпоху потребует много эпох. Компромисс это сделать количество итераций и зармер батча примерно равными. Мы уже смотрели какой длины у нас датасет (`examples_per_epoch`). Корень из него примерно равен 64. Это и будет размер батча.

Для создания батчей используется тот же самый способ, что и при разбиении текста на строки. Тогда от одномерного тензора мы перещли к двумерному (потом мы еще каждый его элемент сделали парой строк), а теперь перейдём к трёхмерному тензору. Это будут пары двумерных массивом чисел, где по первому измерению будет индекс строки, а по второму измерению идут числа, которые являются индексом символа в нашем словаре. 
Также сейчас пары строк идут одна за одной, а лучше их перемешать. Для этого тоже есть готовые методы объекта tf.data.Dataset.
Еще надо будет выбрать размер буфера для перемешивания датасета на ходу. Подробней можно почитать в документации к классу tf.data.Dataset и его методу shuffle.

In [ ]:
# Check dataset classes
print("dataset class is {}".format(dataset))
print("dataset is a child of tf.data.Dataset? {}.".format(isinstance(dataset, tf.data.Dataset)))

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset_batched = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print("dataset_batched is new variable of class {}.".format(dataset_batched))
print("dataset_batched is a child of tf.data.Dataset? {}.".format(isinstance(dataset_batched, tf.data.Dataset)))

Попробуем снова визуализировать наши символы в датасете из пар массивов строк для входа и выхода.

In [ ]:
# Take first dataset batch as a pair
for input_batch, target_batch in dataset_batched.take(1):
    pass

# You can check that we have 2D array with the following debug output
print("input_batch is new variable of class {}.".format(input_batch))
print("So dataset_batched can be converted to a pair of arrays with shape {} and {}.".format(input_batch.shape, target_batch.shape))

# Let's look into some strings
for i in range(2):
    input_example = input_batch[i]
    target_example = target_batch[i]
    print("String pair #{}".format(i))
    show_str(input_example, "Input sequence:")
    show_str(target_example, "Target sequence:")

Каждый раз, когда мы перезапускаем предыдущий блок, мы получаем разные строки. Это отложенные вычисления. Наш `dataset_batched` не хранит массивы строк в случайном порядке. Он каждый раз порождает по 64 строки длиной 100 символов.

## Построим модель сети

Снова будем использовать модель Sequential `tf.keras.Sequential`. В ней сделаем 3 ключевых слоя. Первый слой нужен для конвертации индекса символа по нашему словарю в более естественное представление для нейронной сети. Например 35 и 36 это очень похожие 2 сигнала, но совершенно разные символы. Используем для такой конвертации one-hot encoding. Встроенного слоя в tensorflow нет, поэтому сделаем свой через Lambda слой. Средним слоев сделаем LSTM (или можно использовать другой RNN нейрон - GRU). Это сделает сеть рекуррентной и позволит сети работать с последовательностями наших объектов: символов-цифр. Последним слоем сделаем обычный Dense с количеством выходов, равным количеству символов в нашем словаре - `vocab_size`, а также привычная активация softmax. Это позволит несложно иметь вероятности следующего символа.

Проверим как работает `tf.one_hot`. На входе должен быть индекс, а на выходе массив с нулями и единицей в месте этого индекса. Мы подадим на вход сразу батч, который двухмерный тензор. Значит на выходе получим трёхмерный тензор.

In [ ]:
for input_batch, target_batch in dataset_batched.take(1):
    pass
tf.one_hot(input_batch, len(vocab))

Теперь зададим параметры модели в одном месте и саму модель. Только заранее предусмотрим, что модель, которая принимает батч и модель, которая принимает одну строку это разные модели. А мы хотим учиться по батчу, а потом использовать (inference) с одной строкой. Поэтому у нас будут 2 модели, отличающиеся размером батча. Веса у них будут теми же самыми, меняется только размер входного тензора. Поэтому мы будем создавать наши слегка отличающиеся модели через функцию. Ну и сразу зададим глобальные параметры.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# Number of RNN units
lstm_units = 1024

def build_model(vocab_size, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # To call tf.one_hot() we use Lambda, but we also have to cast type to uint8 
    # because otherwise model weights can't be imported as they do not support default float32 type.
    # We also have to define batch shape because model can't be used wighout knowing its input shape.
    # It can be defined on first use, but it is more simple to define it now.
    tf.keras.layers.Lambda(lambda x: tf.nn.embedding_lookup(tf.cast(x, 'uint8'), vocab_size), 
                           batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
  ])
  return model

In [ ]:
model_batched = build_model(
  vocab_size = len(vocab),
  rnn_units=lstm_units,
  batch_size=BATCH_SIZE)

model_batched.summary()

Модель, принимающая батчи, создана и уже может быть использована. Естественно, что мы будем ожидать от неё случайные символы, но мы сможем проверить это и посчитать функцию потерь. Видно, что у модели довольно много коэффициентов, а также, что она готова принимать строки любой длины, но самих строк должно быть `BATCH_SIZE`.

## Запустим модель без обучения

Возьмём один элемент нашего датасета (он каждый раз случайный, мы это обсуждали выше). В нём батч из входных и выходных строк. Посмотрим их размерности, а также заставим сеть по входной строке сгенерировать выходную и посмотрим её размерности.

In [ ]:
for input_example_batch, target_example_batch in dataset_batched.take(1):
  print("Input shape ", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target shape ", target_example_batch.shape, "# (batch_size, sequence_length)")
  example_batch_predictions = model_batched(input_example_batch)
  print("Prediction shape ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Сеть выдаёт батч из строк той же длины, что и входные строки, но вместо символов у нас массивы чисел с длиной `vocab_size`. Всё верно. Нужно выбрать из них символ, который сеть считает следующим. Используем для этого argmax. Возьмём нулевую строку из батча и укажем, что Argmax надо брать вдоль последнего измерения. В итоге мы должны получить строку из 100 индексов массива.

In [ ]:
sampled_indices = tf.argmax(example_batch_predictions[0], axis=1)
print(sampled_indices)

Это строка индексов нашего словаря, которая была сгенерирована посимвольно. Сначала мы подали всего один символ и считали символ на выходе. Поэтому даже обученная сеть на данном этапе видела только один символ. Обученная сеть на первый символ "I" наверное бы догадалась поставить пробел, так как это наиболее частая последовательность. Но у нас сеть необученная, поэтому она будет генерировать случайные символы.

Можно будет перезапустить этот блок после обучения и посмотреть, как сеть пытается генерировать символы строки. Строка на входе и строка на выходе должны быть похожи. При этом важно понимать, что каждый символ предсказанной строки продолжает исходную входную строку, а не выходную. Удачные предсказания это совпадение символов на одинаковых позициях target и predicted строк. После долгого обучения сеть начинает где-то с 10 буквы вспоминать предложение и выдавать его в точности равной target строке.

In [ ]:
show_str(input_example_batch[0], "Input sequence:")
show_str(target_example_batch[0], "Target sequence:")
show_str(sampled_indices, "Predicted sequence:")

## Обучение сети

Итак, мы решаем стандартную проблему классификации. Для этого используем стандартную функцию потерь `tf.keras.losses.sparse_categorical_crossentropy` (она будет применяться к последней размерности выходного тензора) и стандартный оптимизатор `adam`.

In [ ]:
model_batched.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy)

### Проверка перед обучением
Мы можем посчитать потери вручную для необученной сети. С этого значения начнётся обучение. Если сеть уже была обучена, то запуск этого блока покажет потери, на которых обучение закончилось и с которых его можно продолжить.

In [ ]:
example_batch_loss  = tf.keras.losses.sparse_categorical_crossentropy(target_example_batch, example_batch_predictions)
print("scalar_loss: ", example_batch_loss.numpy().mean())

### Запуск обучения
Введём количество эпох для обучения и запустим его. Обучение 10 эпох занимает разумное время для получения какого-то результата. Для получения хорошего результата лучше учиться 30 и больше эпох.

In [ ]:
EPOCHS=30

In [ ]:
history = model_batched.fit(dataset_batched, epochs=EPOCHS)

## Генерация текста (inference)
Пришло время взять обученную сеть и применить её для генерации. Для этого нужно будет решить 2 проблемы: переход к сети с размером батча 1, а не `BATCH_SIZE`; а также использование сети именно для генерации текста, а не продолжении его на одну букву. Последнее мы сделаем рекурсивно.

### Переход к размеру батча 1
Напомним, что если обучение плохо делать по одной строке, то использование по одной строке делать удобно. И для перехода не нужно переобучать сеть. Нужно создать такую же сеть, только с другим батчем, а потом воспользоваться сохранением и загрузкой весов, как описано в https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/

Перед использованием модель надо построить или скомпилировать.

In [ ]:
model_weights = model_batched.get_weights()

model_single = build_model(vocab_size, lstm_units, batch_size=1)

model_single.set_weights(model_weights)

model_single.build()

Проверим, что модель изменилась и принимает 1 строку.

In [ ]:
model_single.summary()

### Рекурсивная генерация

Начинаем со стартовой строки. Она может быть любой длины, но именно её сеть должна продолжать. Поэтому стоит ввести хотя бы несколько слов.

Входную строку мы должны преобразовать в индекс словаря, затем дать ей еще одно измерение, чтобы это был батч размера 1. А в генерируемой строке мы должны убрать назад лишнее измерение от батча, взять вектор выхода и выбрать наилучший символ, как argmax и преобразовать из индекса массива назад в символ по словарю.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = np.array([char2idx[s] for s in start_string])
  print("input_eval original shape is ", input_eval.shape)
  input_eval = tf.expand_dims(input_eval, 0)
  print("input_eval shape changed to ", input_eval.shape)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  # We reset the memory of RNN so it will forget the previous timeline os characters sequence
  model.reset_states()
    
  for i in range(num_generate):
      predictions = model(input_eval)
      # print("Predictions shape before squeezing ", predictions.shape)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      # print("Predictions shape after squeezing ", predictions.shape)

      # using argmax() to determine next symbol
      temperature = 1
      matrix = np.reshape(predictions[0], (1, -1))
      predicted_id = tf.random.categorical(tf.math.log(matrix), 1)
      predicted_id = predicted_id[0]
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model_single, start_string=u"We are on the verge of "))

Текст генерируется, но зациклился? Причина в том, что самый вероятный символ продолжает какую-то фразу, где может встретиться предыдущая короткая последовательность символов, что снова сделает вероятным этот символ. Например, "Я хочу сказать, что, это, как его, ну это, как его, ну это, как его...". При этом, если мы запустим блок проверки, который мы ранее запускали на необученной сети, то он работает сносно. Ему на вход поступает строка из нашего датасета. А там нет зацикливания.

Способов разорвать цикл 2: более долгая память (более долгое обучение), следование не всегда самому вероятному символу. Долгая память строит более длинные связные фразы и вероятность цикла снижается. Можешь попробовать поучить сеть еще 10 эпох и длина цикла скорее всего вырастет. Второй способ - следование не всегда самому вероятному символу, а другому из вероятных, позволяет выходить из цикла.

### Эксперимент 1
Чтобы разорвать цикличность текста нужно выбирать не самый вероятный символ, а один из вероятных. У нас же на выходе вектор вероятностей, так давайте им и воспользуемся. Для этого существует функция `tf.random.categorical`. В документации про неё немного сказано, но можно проверить как она действует.

In [ ]:
num_of_samples = 200
probabilities = np.array([[0.5, 0.2, 0.2, 0.1]])
temperature = 1

samples = tf.random.categorical(tf.math.log(probabilities) / temperature, num_of_samples)
print(probabilities.shape)

На входе матрица вероятностей (нужна именно матрица), а также сколько индексов нагенерировать согласно этим вероятностям. Также обрати внимание на логарифм. Проверим, что статистика генерации совпадает с вероятностями. Для этого воспользуемся нашим стандартным matplotlib.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# the histogram of the data
_, bins, _ = plt.hist(samples, bins=len(probabilities[0]), 
                            density=False, color='lawngreen', rwidth = 0.8)
# Probability lines
for p in probabilities[0]:
    plt.plot(bins, [p*num_of_samples for _ in bins], color='b' )
plt.show()

Количество сгенерированных элементов примерно равняется нужному проценту от общего количества. Если не совсем это видно, то можно поднять количество `num_of_samples`.
### Задание
Примени выбор элемента согласно выдаваемым сетью вероятностям вместо `argmax` и исправь зацикленность текста. При этом ты можешь регулировать разброс вероятностей с помощью `temperature`. Нулевая температура по сути эквивалентна argmax, а бесконечная температура просто выдаёт случайный символ.

### Эксперимент 2
Повысим скорость обучения сети и одновременно упростим её за счёт избавления от вручную созданного слоя. Вместо жесткой кодировки каждого символа в one-hot вектор можно использовать эмбеддинги. А именно кодировку индекса в вектор, где сложие объекты имеют сонаправленные значения, а противоположные объекты по смыслу будут противонаправленные. Эмбеддинги в основном используют для слов, а не для символов, но и для символов они подходят. Например пунктуация сложа между собой, а символы от них отличны. Проще всего понять как должны работать эмбеддинги можно через пример. Должно примерно выполняться следующее равенство векторов из эмбеддингов слов "Король" - "Мужчина" + "Женщина" ~= "Королева".
### Задание
Примени слой эмбеддингов вместо слоя one-hot для символов, найдя его в документации и подобрав размерность эмбеддинга. Запиши насколько упали потери после 10 эпох обучения по сравнению с прошлой сетью со слоем one-hot. Не меняй сеть обратно, раз с эмбеддингами работает лучше.